<a href="https://colab.research.google.com/github/shonyeajin/ballet_pose_superviser/blob/main/annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

input directory읽어서 모든 image 파일을 json으로 변환해서 output directory에 저장함
잘 되었는지 확인하기 위해서 이미지 위에 합성해서 보여줌

In [9]:
%cd '/content/drive/MyDrive/Colab Notebooks/vision_project/input'

/content/drive/MyDrive/Colab Notebooks/vision_project/input


In [23]:
import numpy as np
import cv2
import sys
import os
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow


In [33]:
protoFile="/content/drive/MyDrive/Colab Notebooks/vision_project/model/pose_deploy.prototxt"
weightsFile="/content/drive/MyDrive/Colab Notebooks/vision_project/model/pose_iter_584000.caffemodel"
nPoints=25
BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]
def proc(img):
  global protoFile, weightsFile, nPoints, BODY_PARTS_BODY_25

  print("./input/"+img)
  frame=cv2.imread("/content/drive/MyDrive/Colab Notebooks/vision_project/input/"+img)
  cv2_imshow(frame)
  frameCopy=np.copy(frame)

  image_height=368
  image_width=368

  input_blob=cv2.dnn.blobFromImage(frame, 1.0/255, (image_width, image_height), (0,0,0), swapRB=False ,crop=False)
  net=cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
  net.setInput(input_blob)
  out=net.forward()

  out_height=out.shape[2]
  out_width=out.shape[3]

  frame_height, frame_width = frame.shape[:2]

  points=[]

  for i in range(len(BODY_PARTS_BODY_25)):
    prob_map=out[0,i,:,:]
    min_val, prob,min_loc, point=cv2.minMaxLoc(prob_map)
    x=(frame_width * point[0])/out_width
    x=int(x)
    y=(frame_height * point[1])/out_height
    y=int(y)

    if prob> 0.2:
      cv2.circle(frameCopy, (x,y),5, (0,255,255), thickness=-1, lineType=cv2.FILLED)
      cv2.putText(frameCopy, str(i), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 1, lineType=cv2.LINE_AA)
      points.append((x,y))
    else:
      cv2.circle(frameCopy, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
      cv2.putText(frameCopy, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)
      points.append(None)
  cv2_imshow(frameCopy)

    

In [ ]:
for i in os.listdir("/content/drive/MyDrive/Colab Notebooks/vision_project/input"):
  if i[-3:] in ['jpg', 'png', 'peg']:
    proc(i)